In [1]:
import os 
import numpy as np 
import pickle
from tqdm import tqdm 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import copy


from sklearn.utils import resample, shuffle
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import PredefinedSplit
from scipy.optimize import curve_fit


In [2]:
# functions 
###### helper functions ####### 
def pkl_read(file_to_read, write_dir):
  this = pickle.load(open(os.path.join(write_dir,file_to_read), "rb"))
  # print(this)
  return this

def pkl_write(file_to_write, values_to_dump, write_dir):
  os.chdir(write_dir)
  with open(os.path.join(write_dir,file_to_write), 'wb') as pickle_file:
      pickle.dump(values_to_dump, pickle_file)

def getdata_day(data,id,data_day):
  if id == 1:
    # var_name = data[0:data_day[id]+1,:,:]
    var_name = data[0:data_day[id],:,:]
  else:
    day_idx = np.where(np.array(list(data_day))== id)[0].astype(int) - 1
    #print(day_idx[0])
    prev_day_id = list(data_day)[day_idx[0]]
    #print(prev_day_id)
    var_name = data[data_day[prev_day_id]:data_day[id],:,:]
  return var_name

def gettarget_day(target_direction, id, day):
    if id == 1: 
      target = target_direction[:day[id]]
    else:
      day_idx = np.where(np.array(list(day))== id)[0].astype(int) - 1
      #print(day_idx[0])
      prev_day_id = list(day)[day_idx[0]]
      target = target_direction[day[prev_day_id]:day[id]] 
    return target

def get_numtrials_perday(datasize_day):
    '''
    datasize_day: dictionary with key as day and value as end trial index for that day . Comes from getdata_day()
    '''
    num_trials_perday = np.zeros(len(datasize_day))
    num_trials_perday[0] = np.array(list(datasize_day.values()))[0]
    num_trials_perday[1:] = np.diff(np.array(list(datasize_day.values())))
    # print(num_trials_perday)
    return num_trials_perday

def trial_concatenate_data(data):
    '''
    data is on shape n_trials x n_units x n_bins 
    '''
    n,u,t = data.shape
    new_arr = np.transpose(data, (0,2,1))
    new_arr = new_arr.reshape(n*t, u)
    # print(new_arr.shape)
    return new_arr 

###### LR analysis functions #########

def get_equal_target_dist_predefinedsplit(idx, target, n_samples_per_target):
  random_idx = []
  for it in range(8):
    this_idx = np.where(target== it+1)
    
    r_idx = np.random.choice(this_idx[0], n_samples_per_target, replace = False)
    # print(r_idx)
    random_idx.append(r_idx)
  # print(np.shape(random_idx))
  rand_idx = np.array(random_idx).reshape(n_samples_per_target*8)
  ps = np.zeros(len(target))
  ps[rand_idx] = -1 # index for training 
  # ps[ps == 0] = 1 # index for test set

  # to genereate 2 fold cross validation
  test_idx = np.argwhere(~np.in1d(np.arange(len(ps)), rand_idx))
  first_fold = test_idx[:len(test_idx)//2]
  second_fold = test_idx[len(test_idx)//2:]
  ps[first_fold] = 0   # index for test set for fold 1
  ps[second_fold] = 1 # index for test set for fold 2
  print(ps[ps == -1].shape, ps[ps == 0].shape, ps[ps == 1].shape)
  return ps

def flatten_data(data):
    # Flattens the last two dimensions of the data
    return data.reshape(data.shape[0], -1)


def matrix_similarity(A, B):
    """
    Compute the R^2 similarity between two matrices by treating them as vectors.

    Args:
    - A: First 2D matrix.
    - B: Second 2D matrix.

    Returns:
    - R^2 value representing the similarity between the matrices.
    """
    assert A.shape == B.shape, "Matrices must have the same shape"

    # Flatten the matrices to vectors
    A_flat = A.ravel()
    B_flat = B.ravel()

    # Compute the correlation coefficient
    correlation_matrix = np.corrcoef(A_flat, B_flat)
    r = correlation_matrix[0, 1]
    
    # Square the correlation coefficient to get R^2 value
    r2 = r #**2
    return r2



In [4]:
neural_recordings = pkl_read('neural_data.pkl', './example_data/') # Shape: n_tr x n_units x n_timebins; top 16 units are readouts 
target_direction = pkl_read('target_labels.pkl', './example_data/')
datasize_day = pkl_read('trial_day_label.pkl', './example_data/')

days = [2,3,4,5,6,7] # just for example. In the paper, we considered all days that had a minimum of 25 trials of successful reaches per target direction, minimum 200 trials in total.  
n_readouts = 20
decoder_change_days = [0,0,0,0,0,1]
unit_swap_day = [0,0,0,0,0,0]